In [1]:
import pdfplumber
import re
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
pdf = pdfplumber.open("Old_city.pdf")
all_pages = pdf.pages
cleaned_lines = []
cleaned_list_of_dicts = []
for each_page in all_pages:
    text_of_each_page = each_page.extract_text()
    all_lines = text_of_each_page.splitlines()
    for each_line in all_lines:
        if each_line == "Address":
            continue
        elif each_line == "Owner Card  Mailing Label  Map and Details Report":
            continue
        elif each_line == "KGIS Parcel Report":
            continue
        else:
            cleaned_lines.append(each_line)
each_dict = {}
for index, cleaned_line in enumerate(cleaned_lines):
    if re.match(r"^09[0-9A-Z]{5,}$", cleaned_line):
        # parcel_id
        each_dict["parcel_id"] = cleaned_line
        each_dict["name"] = cleaned_lines[index + 1]
        cleaned_list_of_dicts.append(each_dict)
        each_dict = {}
    else:
        continue

In [3]:
len(cleaned_list_of_dicts)

336

In [ ]:
# cleaned_list_of_dicts

In [17]:
driver = webdriver.Chrome(ChromeDriverManager().install())

/var/folders/2g/fbs070qj3qd3kw4ntm252trc0000gp/T/ipykernel_90668/1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [18]:
def get_all_data(parcel_number, agree):
    driver.get(
        "https://propertyinfo.knoxcountytn.gov/search/commonsearch.aspx?mode=realprop"
    )
    if agree:
        try:
            driver.find_element(By.ID, "btAgree").click()
        except:
            print("No agree modal found")
    driver.find_element(By.ID, "inpParid").send_keys(parcel_number)
    driver.find_element(By.ID, "btSearch").click()
    error_passing = driver.find_elements(By.ID, "searchResults")
    # print(error_passing)
    if len(error_passing) > 0:
        error_passing[0].find_element(By.CLASS_NAME, "SearchResults").click()
    driver.find_element(By.XPATH, '//*[@id="sidemenu"]/ul/li[3]/a').click()
    parcel_id_dirty = driver.find_element(By.CLASS_NAME, "DataletHeaderTop").text
    address_data = driver.find_elements(By.CLASS_NAME, "DataletHeaderBottom")
    parcel_id = parcel_id_dirty.split(": ")[1]
    sales_data = []
    print("address", address_data)
    try:
        sales_details = driver.find_element(By.ID, "Sales Summary")
        # print("about to rows")
        rows = sales_details.find_elements(By.TAG_NAME, "tr")[1:]
        # print("about to row")
        for row in rows:
            all_data = row.find_elements(By.TAG_NAME, "td")
            # print("all_data", all_data)
            if len(all_data) < 5:
                # print("insufficient data")
                continue
            else:
                data = {}
                date = all_data[0].text
                # print("date done")
                book = all_data[1].text
                # print("book done")
                page = all_data[2].text
                # print("page done")
                price = all_data[3].text
                # print("price done")
                grantee = all_data[-1].text
                # print("grantee done")
                address = address_data[-1].text
                # print("address done")

                data["date"] = date
                data["book_number"] = book
                data["page"] = page
                data["sale_price"] = price
                data["grantee_name"] = grantee
                data["parcel_id_new"] = parcel_id
                data["address"] = address
                sales_data.append(data)
    except Exception as e:
        print(e)
        data = {}
        data["date"] = "Not found"
        data["book_number"] = "Not found"
        data["page"] = "Not found"
        data["sale_price"] = "Not found"
        data["grantee_name"] = "Not found"
        data["parcel_id_new"] = "Not found"
        data["address"] = "Not found"
        sales_data.append(data)
    return sales_data

In [6]:
all_sales_data = []

In [19]:
agree = True
for index, each_parcel_dict in enumerate(cleaned_list_of_dicts[301:]):
    sales_data = get_all_data(each_parcel_dict["parcel_id"], agree)
    agree = False
    all_sales_data.extend(sales_data)
#     print(index, "done")

address [<selenium.webdriver.remote.webelement.WebElement (session="0cc014dd70888029971e47215caadd8c", element="f12b6be2-c2f5-4fb1-ab36-f518e09aa3cf")>, <selenium.webdriver.remote.webelement.WebElement (session="0cc014dd70888029971e47215caadd8c", element="ba12ce58-7e71-4ec6-bec0-e743aaaabcc6")>, <selenium.webdriver.remote.webelement.WebElement (session="0cc014dd70888029971e47215caadd8c", element="7dc4b255-5540-4c8f-9803-279a3c1aae50")>]
address [<selenium.webdriver.remote.webelement.WebElement (session="0cc014dd70888029971e47215caadd8c", element="4929cdb5-2622-47cc-9870-9f6dd96e29b6")>, <selenium.webdriver.remote.webelement.WebElement (session="0cc014dd70888029971e47215caadd8c", element="1f91bfde-7a9d-4aef-a21a-2325e1f64c9b")>, <selenium.webdriver.remote.webelement.WebElement (session="0cc014dd70888029971e47215caadd8c", element="0f4b350a-75a6-4578-94c6-4b6c98caaa45")>]
address [<selenium.webdriver.remote.webelement.WebElement (session="0cc014dd70888029971e47215caadd8c", element="83504a

address [<selenium.webdriver.remote.webelement.WebElement (session="0cc014dd70888029971e47215caadd8c", element="032f9963-6573-4563-a1cc-d4fce9ae0542")>, <selenium.webdriver.remote.webelement.WebElement (session="0cc014dd70888029971e47215caadd8c", element="65071f8e-44e7-4937-ac16-c0e10cce5b8b")>, <selenium.webdriver.remote.webelement.WebElement (session="0cc014dd70888029971e47215caadd8c", element="c6352c31-3226-4d50-9937-06d5545a7595")>]
address [<selenium.webdriver.remote.webelement.WebElement (session="0cc014dd70888029971e47215caadd8c", element="5630221d-00c1-4868-a39e-9b1710831d48")>, <selenium.webdriver.remote.webelement.WebElement (session="0cc014dd70888029971e47215caadd8c", element="4252cd57-c088-43a3-b7b1-ddfe51fb7a3f")>, <selenium.webdriver.remote.webelement.WebElement (session="0cc014dd70888029971e47215caadd8c", element="ac7f7103-b5b9-46cd-919e-cc68b923c868")>]
address [<selenium.webdriver.remote.webelement.WebElement (session="0cc014dd70888029971e47215caadd8c", element="ebaa11

In [20]:
len(all_sales_data)

1275

In [21]:
pd.set_option('display.max_columns', None)
old_city = pd.DataFrame(all_sales_data)

In [22]:
old_city

,date,book_number,page,sale_price,grantee_name,parcel_id_new,address
0,12/08/1998,00002309,0000728,"$450,000",HEURISTIC WORKSHOP INC,094EF043,203 JACKSON AVE W
1,07/19/2022,20220721,0004903,"$4,000,000",EPHANT GROUP INC,095HA001,130 CENTRAL ST S
2,04/14/2022,20220419,0080051,"$2,400,000",GP CORNER LLC,095HA001,130 CENTRAL ST S
3,01/29/2021,20210201,0062283,"$558,468",WAYLAND-GOODMAN PROPERTIES L P,095HA001,130 CENTRAL ST S
4,12/17/2004,20041217,0049650,$0,WAYLAND- GOODMAN PROPERTIES L P,095HA001,130 CENTRAL ST S
...,...,...,...,...,...,...,...
1270,10/31/2001,20011101,0034821,"$700,000",JACKSON AVENUE PARTNERSHIP LLC,095HA016,200 JACKSON AVE E
1271,05/11/1992,00002082,0000254,$0,BRIGHT-KAPLAN INTERNATIONAL CORPORATION,095HA016,200 JACKSON AVE E
1272,05/06/1992,00002071,0000534,"$225,000",BRIGHT-KAPLAN INTERNATIONAL CORPORATION,095HA016,200 JACKSON AVE E
1273,09/23/1983,00001798,0000120,"$50,000",SHARNAN INC,095HA016,200 JACKSON AVE E


In [23]:
old_city.to_csv('old-city.csv')